In [6]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from IPython.display import clear_output
from itertools import product
from src.loading import *
from src.saving import *
from src.moisture_space import *
import matplotlib.pyplot as plt
import matplotlib.colors as colors

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Phase Composites

In [18]:
# Get files for raw data
#
variable = 'cli'
variable_files = get_raw_gsam_variable_files(variable)
# Get satfrac files
#
satfrac_files = get_daily_combined_2d_gsam_files()
assert(len(variable_files)==len(satfrac_files))

# Load PCs
#
pcs = load_gsam_eofs_pcs().scores()
pcs = pcs/pcs.std(('lat', 'lon', 'time'))
# Compute phase angle, phase number, and radius
# Where radius < 0.5, we set phase number to 0
#
theta = np.arctan2(pcs.sel(mode=2), pcs.sel(mode=1))
radius = np.sqrt(pcs.sel(mode=2)**2 + pcs.sel(mode=1)**2)
phase_num = theta.copy(data=np.floor(np.mod((np.array(theta)+(np.pi/8))/(np.pi/4) + 4, 8))+1)
phase_num = phase_num.where(radius > 0.5, other=0, drop=False)

# The extracted grids will be stored in a dictionary.
# Both the variable data and the saturation fraction will be kept
#
sorted_var_grids_by_phase = {f'phase{n}': [] for n in range(1, 9)}

for i, (vf, sff) in enumerate(zip(variable_files, satfrac_files)):
    clear_output()
    print(f'File {i+1} of {len(variable_files)}')
    var_ds = xr.open_dataset(vf)[variable]
    satfrac_ds = xr.open_dataset(sff)
    satfrac_ds = satfrac_ds['PW']/satfrac_ds['PWS']
    
    # Segment the data into grids
    #
    gridded_var = segment_data_into_grids(var_ds, patch_length=50)
    gridded_satfrac = segment_data_into_grids(satfrac_ds, patch_length=50)

    for phase in range(1, 9):
        # Identify which large-scale grids are in the desired phase
        #
        phase_mask = (phase_num.sel(time=gridded_var.time)==phase)
        (grid_time_idx, grid_lat_idx, grid_lon_idx) = np.where(phase_mask)
        phase_grids = [
            gridded_var
            .isel({'time': time_idx})
            .sel({'coarse_grid_lat': lat_idx, 'coarse_grid_lon': lon_idx})
            .stack(column=('lat', 'lon'))
            for (time_idx, lat_idx, lon_idx) in zip(grid_time_idx, grid_lat_idx, grid_lon_idx)
        ]
        
        phase_satfrac = [
            gridded_satfrac
            .isel({'time': time_idx})
            .sel({'coarse_grid_lat': lat_idx, 'coarse_grid_lon': lon_idx})
            .stack(column=('lat', 'lon'))
            for (time_idx, lat_idx, lon_idx) in zip(grid_time_idx, grid_lat_idx, grid_lon_idx)
        ]

        # Sort each grid by its sat frac values
        #
        sorted_var_grids_by_phase[f'phase{phase}'].extend(
            [ 
                grid
                .sortby(satfrac)
                .drop_vars(['column', 'lat', 'lon'])
                .assign_coords({'column': np.linspace(0, 1, grid.column.size)})
                for (grid, satfrac) in zip(phase_grids, phase_satfrac)
            ]
        )

# Composite the grids in each phase
# We compute
# - Composite: average over all data
# - Composite anomaly: average of anomalies for each grid
# - Composite mean: average of the averages (average large-scale profiles)
#
composite_grids = {phase: None for phase in sorted_var_grids_by_phase.keys()}
composite_anomaly_grids = {phase: None for phase in sorted_var_grids_by_phase.keys()}
composite_mean_grids = {phase: None for phase in sorted_var_grids_by_phase.keys()}

for phase, grids in sorted_var_grids_by_phase.items():
    print(f'Processing Phase {phase}')
    phase_grids = xr.concat(
        grids,
        dim=pd.Index(range(len(grids)), name='member')
    )
    
    phase_grid_means = phase_grids.mean('column')
    phase_grid_anoms = phase_grids - phase_grid_means
    composite_grids[phase] = phase_grids.mean('member')
    composite_mean_grids[phase] = phase_grid_means.mean('member')
    composite_anomaly_grids[phase] = phase_grid_anoms.mean('member')

# Save the grids
#
for p in range(1, 9):
    print(f'Saving Phase {p}...')
    composite_grids[f'phase{p}'].to_netcdf(get_project_data_dir()+f'phase_composites/phase{p}_composite_{variable}.nc')
    composite_anomaly_grids[f'phase{p}'].to_netcdf(get_project_data_dir()+f'phase_composites/phase{p}_composite_anomaly_{variable}.nc')
    composite_mean_grids[f'phase{p}'].to_netcdf(get_project_data_dir()+f'phase_composites/phase{p}_composite_mean_{variable}.nc')

File 29 of 29
Processing Phase phase1
Processing Phase phase2
Processing Phase phase3
Processing Phase phase4
Processing Phase phase5
Processing Phase phase6
Processing Phase phase7
Processing Phase phase8
Saving Phase 1...
Saving Phase 2...
Saving Phase 3...
Saving Phase 4...
Saving Phase 5...
Saving Phase 6...
Saving Phase 7...
Saving Phase 8...


# Composites of Different Evolutions

In [8]:
# Get files for raw data
#
variable = 'cli'
variable_files = get_raw_gsam_variable_files(variable)
# Get satfrac files
#
satfrac_files = get_daily_combined_2d_gsam_files()
assert(len(variable_files)==len(satfrac_files))

# Load PCs
#
pcs = load_gsam_eofs_pcs().scores()
pcs = pcs/pcs.std(('lat', 'lon', 'time'))
# Compute phase angle, phase number, and radius
# Where radius < 0.5, we set phase number to 0
#
theta = np.arctan2(pcs.sel(mode=2), pcs.sel(mode=1))
radius = np.sqrt(pcs.sel(mode=2)**2 + pcs.sel(mode=1)**2)
phase_num = theta.copy(data=np.floor(np.mod((np.array(theta)+(np.pi/8))/(np.pi/4) + 4, 8))+1)
phase_num = phase_num.where(radius > 0.5, other=0, drop=False)

# At this stage we logically index which grids undergo what kind of transition
# We have to do it here because the actual variable data is cut up by day
#
initial_phase = phase_num.isel(time=slice(None, -1)).data
final_phase = phase_num.isel(time=slice(1, None)).data
raw_delta = final_phase - initial_phase
delta_signed = ((raw_delta + 4) % 8) - 4  # this accounts for the 8 to 1 transition while keeping negative numbers

pad = np.full((1, *phase_num.isel(time=0).shape), np.nan)  # since transitions from the last time step can't be determined
delta_phase_data = np.concatenate([delta_signed, pad], axis=0)
delta_phase = phase_num.copy(data=delta_phase_data)

# The extracted grids will be stored in a dictionary.
# Both the variable data and the saturation fraction will be kept
#
sorted_var_grids_by_phase_and_transition = {
    f'phase{n}_{trans_type}': [] for n,trans_type in product(range(1, 9), ['fw', 'bw', 'st'])
}

for i, (vf, sff) in enumerate(zip(variable_files, satfrac_files)):
    clear_output()
    print(f'File {i+1} of {len(variable_files)}')
    var_ds = xr.open_dataset(vf)[variable]
    satfrac_ds = xr.open_dataset(sff)
    satfrac_ds = satfrac_ds['PW']/satfrac_ds['PWS']
    
    # Segment the data into grids
    #
    gridded_var = segment_data_into_grids(var_ds, patch_length=50)
    gridded_satfrac = segment_data_into_grids(satfrac_ds, patch_length=50)

    # Loop over phases and transition types for this file's times
    #
    for phase, trans_type in product(range(1, 9), ['fw', 'bw', 'st']):
        print(f'Phase {phase} Trans Type {trans_type}')
        # Identify which large-scale grids are in the desired phase
        # AND that undergo the desired phase transition
        #
        phase_mask = (phase_num.sel(time=gridded_var.time)==phase)
        
        if trans_type == 'fw':
            right_delta = 1
        elif trans_type == 'bw':
            right_delta = -1
        elif trans_type == 'st':
            right_delta = 0

        right_phase_and_trans_mask = phase_mask & (delta_phase==right_delta)
        (grid_time_idx, grid_lat_idx, grid_lon_idx) = np.where(right_phase_and_trans_mask)
        phase_grids = [
            gridded_var
            .isel({'time': time_idx})
            .sel({'coarse_grid_lat': lat_idx, 'coarse_grid_lon': lon_idx})
            .stack(column=('lat', 'lon'))
            for (time_idx, lat_idx, lon_idx) in zip(grid_time_idx, grid_lat_idx, grid_lon_idx)
        ]
        
        phase_satfrac = [
            gridded_satfrac
            .isel({'time': time_idx})
            .sel({'coarse_grid_lat': lat_idx, 'coarse_grid_lon': lon_idx})
            .stack(column=('lat', 'lon'))
            for (time_idx, lat_idx, lon_idx) in zip(grid_time_idx, grid_lat_idx, grid_lon_idx)
        ]

        # Sort each grid by its sat frac values
        #
        sorted_var_grids_by_phase_and_transition[f'phase{phase}_{trans_type}'].extend(
            [ 
                grid
                .sortby(satfrac)
                .drop_vars(['column', 'lat', 'lon'])
                .assign_coords({'column': np.linspace(0, 1, grid.column.size)})
                for (grid, satfrac) in zip(phase_grids, phase_satfrac)
            ]
        )

# Composite the grids in each phase undergoing each transition type
# We compute
# - Composite: average over all data
# - Composite anomaly: average of anomalies for each grid
# - Composite mean: average of the averages (average large-scale profiles)
#
composite_grids_by_trans_type = {phase_and_trans_type: None for phase_and_trans_type in sorted_var_grids_by_phase_and_transition.keys()}
composite_anomaly_grids_by_trans_type = {phase_and_trans_type: None for phase_and_trans_type in sorted_var_grids_by_phase_and_transition.keys()}
composite_mean_grids_by_trans_type = {phase_and_trans_type: None for phase_and_trans_type in sorted_var_grids_by_phase_and_transition.keys()}


for phase_and_trans_type, grids in sorted_var_grids_by_phase_and_transition.items():
    print(f'Processing Phase {phase_and_trans_type}')
    phase_grids = xr.concat(
        grids,
        dim=pd.Index(range(len(grids)), name='member')
    )
    
    phase_grid_means = phase_grids.mean('column')
    phase_grid_anoms = phase_grids - phase_grid_means
    composite_grids_by_trans_type[phase_and_trans_type] = phase_grids.mean('member')
    composite_anomaly_grids_by_trans_type[phase_and_trans_type] = phase_grid_anoms.mean('member')
    composite_mean_grids_by_trans_type[phase_and_trans_type] = phase_grid_means.mean('member')


# Save the grids
#
for phase_and_trans_type, grids in sorted_var_grids_by_phase_and_transition.items():
    print(f'Saving {phase_and_trans_type}...')
    composite_grids_by_trans_type[phase_and_trans_type].to_netcdf(get_project_data_dir()+f'phase_composites/by_trans_type/{phase_and_trans_type}_composite_{variable}.nc')
    composite_anomaly_grids_by_trans_type[phase_and_trans_type].to_netcdf(get_project_data_dir()+f'phase_composites/by_trans_type/{phase_and_trans_type}_composite_anomaly_{variable}.nc')
    composite_mean_grids_by_trans_type[phase_and_trans_type].to_netcdf(get_project_data_dir()+f'phase_composites/by_trans_type/{phase_and_trans_type}_composite_mean_{variable}.nc')

File 29 of 29
Phase 1 Trans Type fw
Phase 1 Trans Type bw
Phase 1 Trans Type st
Phase 2 Trans Type fw
Phase 2 Trans Type bw
Phase 2 Trans Type st
Phase 3 Trans Type fw
Phase 3 Trans Type bw
Phase 3 Trans Type st
Phase 4 Trans Type fw
Phase 4 Trans Type bw
Phase 4 Trans Type st
Phase 5 Trans Type fw
Phase 5 Trans Type bw
Phase 5 Trans Type st
Phase 6 Trans Type fw
Phase 6 Trans Type bw
Phase 6 Trans Type st
Phase 7 Trans Type fw
Phase 7 Trans Type bw
Phase 7 Trans Type st
Phase 8 Trans Type fw
Phase 8 Trans Type bw
Phase 8 Trans Type st
Processing Phase phase1_fw
Processing Phase phase1_bw
Processing Phase phase1_st
Processing Phase phase2_fw
Processing Phase phase2_bw
Processing Phase phase2_st
Processing Phase phase3_fw
Processing Phase phase3_bw
Processing Phase phase3_st
Processing Phase phase4_fw
Processing Phase phase4_bw
Processing Phase phase4_st
Processing Phase phase5_fw
Processing Phase phase5_bw
Processing Phase phase5_st
Processing Phase phase6_fw
Processing Phase phase6_bw

# Cloud fraction